# Introduction

Experiment with MLflow while perform an Hyperparameter tuning. Different results of a hyperparemeter sweep would be compare, choosing the best run and register it as a model.

In [3]:
!pip install mlflow

In [4]:
# Import Standard Libraries
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

# Read Data

In [5]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Prepare Data

In [6]:
# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)

In [7]:
# Define model signature
signature = infer_signature(train_x, train_y)

# Model Training

In [8]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    """
    The function train a Keras model using the passed data and parameters
    """

    # Define model architecture
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=np.mean(train_x), variance=np.var(train_x)),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):

        # Train the model
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )

        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        # Return dictionary for Hyperopt
        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [9]:
def objective(params):
    """
    Wrap the objective function for Hyperopt
    """
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result


In [10]:
# Define the hyperparameters space
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

# Hyperparameters Tuning

In [11]:
# Set MLflow experiment
mlflow.set_experiment("wine-quality")

with mlflow.start_run():

    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2024/03/24 17:41:56 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.


Epoch 1/3

46/46 [==============================] - 5s 15ms/step - loss: 12.7401 - root_mean_squared_error: 3.5693 - val_loss: 3.9701 - val_root_mean_squared_error: 1.9925

Epoch 2/3

46/46 [==============================] - 0s 9ms/step - loss: 3.4652 - root_mean_squared_error: 1.8615 - val_loss: 3.3150 - val_root_mean_squared_error: 1.8207

Epoch 3/3

46/46 [==============================] - 0s 9ms/step - loss: 2.8845 - root_mean_squared_error: 1.6984 - val_loss: 2.7213 - val_root_mean_squared_error: 1.6496

12/12 [==============================] - 0s 13ms/step - loss: 2.7213 - root_mean_squared_error: 1.6496

  0%|          | 0/8 [00:12<?, ?trial/s, best loss=?]

2024/03/24 17:42:16 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpcq4wftwk/model, flavor: tensorflow). Fall back to return ['tensorflow==2.15.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 



Epoch 1/3

 12%|█▎        | 1/8 [00:20<02:24, 20.59s/trial, best loss: 1.6496384143829346]

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")



46/46 [==============================] - 1s 8ms/step - loss: 3.4427 - root_mean_squared_error: 1.8554 - val_loss: 1.1583 - val_root_mean_squared_error: 1.0763

Epoch 2/3

46/46 [==============================] - 0s 5ms/step - loss: 0.9083 - root_mean_squared_error: 0.9530 - val_loss: 0.7864 - val_root_mean_squared_error: 0.8868

Epoch 3/3

46/46 [==============================] - 0s 4ms/step - loss: 0.7948 - root_mean_squared_error: 0.8915 - val_loss: 0.7581 - val_root_mean_squared_error: 0.8707

12/12 [==============================] - 0s 3ms/step - loss: 0.7581 - root_mean_squared_error: 0.8707

Epoch 1/3

46/46 [==============================] - 1s 7ms/step - loss: 24.6173 - root_mean_squared_error: 4.9616 - val_loss: 19.6898 - val_root_mean_squared_error: 4.4373

Epoch 2/3

46/46 [==============================] - 0s 5ms/step - loss: 15.4583 - root_mean_squared_error: 3.9317 - val_loss: 12.4179 - val_root_mean_squared_error: 3.5239

Epoch 3/3

46/46 [==============================]